# Camera Event Producer B

This notebook implements a Kafka producer that reads data from `camera_event_B.csv` and sends it to a Kafka topic in batches.

In [1]:
# Import required libraries
import pandas as pd
import json
import time
from kafka import KafkaProducer
from datetime import datetime
import os

In [2]:
# Configuration parameters
KAFKA_BOOTSTRAP_SERVERS = '172.25.240.1:9092'
KAFKA_TOPIC = 'camera_events'
BATCH_INTERVAL = 5  # seconds between each batch
PRODUCER_ID = 'Producer-B'
CSV_FILE = 'camera_event_B.csv'  # Path to the CSV file

# Check if the file exists, otherwise use a default path for testing
if not os.path.exists(CSV_FILE):
    CSV_FILE = '/home/student/Assignment 2/temp_data/data/camera_event_B.csv'  # Adjust path as needed for testing
    if not os.path.exists(CSV_FILE):
        print(f"Warning: {CSV_FILE} not found. Please ensure the file exists.")

In [3]:
# Function to load data from CSV file
def load_data(file_path):
    try:
        df = pd.read_csv(file_path)
        print(f"Successfully loaded {len(df)} records from {file_path}")
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

In [4]:
# Function to initialize Kafka producer
def create_kafka_producer():
    try:
        producer = KafkaProducer(
            bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
            value_serializer=lambda v: json.dumps(v).encode('utf-8'),
            key_serializer=lambda k: k.encode('utf-8') if k else None
        )
        print("Kafka producer initialized successfully")
        return producer
    except Exception as e:
        print(f"Error creating Kafka producer: {e}")
        return None

In [5]:
# Function to process and send data in batches
def process_and_send_batches(producer, df):
    # Get unique batch IDs and sort them
    batch_ids = sorted(df['batch_id'].unique())
    
    for batch_id in batch_ids:
        # Get records for current batch
        batch_df = df[df['batch_id'] == batch_id].copy()
        
        # Add producer information
        batch_df['producer_info'] = PRODUCER_ID
        
        # Add current processing timestamp
        batch_df['processing_timestamp'] = datetime.now().isoformat()
        
        # Extract event date from timestamp
        batch_df['event_date'] = pd.to_datetime(batch_df['timestamp']).dt.date.astype(str)
        
        # Convert to list of dictionaries for sending
        batch_records = batch_df.to_dict('records')
        
        # Send each record in the batch to Kafka
        sent_count = 0
        for record in batch_records:
            try:
                # Use car_plate as the key for partitioning
                key = record['car_plate']
                # Send to Kafka topic
                producer.send(KAFKA_TOPIC, key=key, value=record)
                sent_count += 1
            except Exception as e:
                print(f"Error sending record: {e}")
        
        # Flush to ensure all messages are sent before waiting
        producer.flush()
        
        print(f"Sent batch {batch_id} with {sent_count}/{len(batch_records)} records from {PRODUCER_ID}")
        
        # Wait for the specified interval before sending the next batch
        time.sleep(BATCH_INTERVAL)

In [6]:
# Main execution function
def run_producer():
    # Load data from CSV
    df = load_data(CSV_FILE)
    if df is None:
        print("Failed to load data. Exiting.")
        return
    
    # Create Kafka producer
    producer = create_kafka_producer()
    if producer is None:
        print("Failed to create Kafka producer. Exiting.")
        return
    
    try:
        # Process and send data in batches
        print(f"Starting to send data from {PRODUCER_ID} at {datetime.now()}")
        process_and_send_batches(producer, df)
        print(f"Finished sending all data from {PRODUCER_ID} at {datetime.now()}")
    except KeyboardInterrupt:
        print("Producer interrupted by user")
    except Exception as e:
        print(f"Error in producer: {e}")
    finally:
        # Close producer
        if producer is not None:
            producer.close()
            print("Kafka producer closed")

In [ ]:
# Run the producer
if __name__ == "__main__":
    run_producer()

Successfully loaded 556340 records from /home/student/Assignment 2/temp_data/data/camera_event_B.csv
Kafka producer initialized successfully
Starting to send data from Producer-B at 2025-05-29 10:08:07.278724
Sent batch 1 with 5/5 records from Producer-B
Sent batch 2 with 1/1 records from Producer-B
Sent batch 3 with 1/1 records from Producer-B
Sent batch 4 with 4/4 records from Producer-B
Sent batch 5 with 4/4 records from Producer-B
Sent batch 6 with 2/2 records from Producer-B
Sent batch 7 with 3/3 records from Producer-B
Sent batch 8 with 1/1 records from Producer-B
Sent batch 9 with 2/2 records from Producer-B
Sent batch 10 with 2/2 records from Producer-B
Sent batch 11 with 1/1 records from Producer-B
Sent batch 12 with 1/1 records from Producer-B
Sent batch 13 with 1/1 records from Producer-B
Sent batch 14 with 1/1 records from Producer-B
Sent batch 15 with 2/2 records from Producer-B
Sent batch 16 with 2/2 records from Producer-B
Sent batch 17 with 1/1 records from Producer-B
S

Sent batch 170 with 1/1 records from Producer-B
Sent batch 171 with 5/5 records from Producer-B
Sent batch 172 with 1/1 records from Producer-B
Sent batch 173 with 1/1 records from Producer-B
Sent batch 174 with 3/3 records from Producer-B
Sent batch 175 with 1/1 records from Producer-B
Sent batch 176 with 2/2 records from Producer-B
Sent batch 177 with 2/2 records from Producer-B
Sent batch 178 with 3/3 records from Producer-B
Sent batch 179 with 4/4 records from Producer-B
Sent batch 180 with 1/1 records from Producer-B
Sent batch 181 with 1/1 records from Producer-B
Sent batch 182 with 3/3 records from Producer-B
Sent batch 183 with 1/1 records from Producer-B
Sent batch 184 with 1/1 records from Producer-B
Sent batch 185 with 1/1 records from Producer-B
Sent batch 186 with 1/1 records from Producer-B
Sent batch 187 with 1/1 records from Producer-B
Sent batch 188 with 2/2 records from Producer-B
Sent batch 189 with 3/3 records from Producer-B
Sent batch 190 with 2/2 records from Pro

Sent batch 341 with 1/1 records from Producer-B
Sent batch 342 with 1/1 records from Producer-B
Sent batch 343 with 1/1 records from Producer-B
Sent batch 344 with 1/1 records from Producer-B
Sent batch 345 with 2/2 records from Producer-B
Sent batch 346 with 1/1 records from Producer-B
Sent batch 347 with 2/2 records from Producer-B
Sent batch 348 with 1/1 records from Producer-B
Sent batch 349 with 1/1 records from Producer-B
Sent batch 350 with 1/1 records from Producer-B
Sent batch 351 with 2/2 records from Producer-B
Sent batch 352 with 3/3 records from Producer-B
Sent batch 353 with 2/2 records from Producer-B
Sent batch 354 with 3/3 records from Producer-B
Sent batch 355 with 2/2 records from Producer-B
Sent batch 356 with 2/2 records from Producer-B
Sent batch 357 with 1/1 records from Producer-B
Sent batch 358 with 1/1 records from Producer-B
Sent batch 359 with 1/1 records from Producer-B
Sent batch 360 with 1/1 records from Producer-B
Sent batch 361 with 2/2 records from Pro